In [ ]:
! pip install gymnasium
! pip install "gymnasium[atari, accept-rom-license]"
! pip install torch
! pip install torchvision
! pip install numpy
! pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.4 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=1b383916769a798989a50318d92eeee656af2e3c3040d8a981a487984f246938
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir drive/MyDrive/logs_Pacman
! mkdir drive/MyDrive/videos_Pacman

In [ ]:
#import ale_py
#import shimmy
import gymnasium as gym
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn import functional as F
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import random
import copy
import time
import pickle
import torchvision
import matplotlib.pyplot as plt
import math
import json


In [ ]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()   
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*7*7 , out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=5)
        )

    def forward(self, x):
        conv_out = self.conv(x)
        x = torch.flatten(conv_out, start_dim=1)
        return self.fc(x)
        

In [ ]:
class Memory():
    
    def __init__(self,size):
        self.size = size
        self.experiences = []
    
    def sample(self,batch_size):
        return random.choices(self.experiences, k=batch_size)
        
    def add(self, experience):
        self.experiences.append(experience)
        if len(self.experiences) > self.size:
            self.experiences.pop(0)
                    
    def __len__(self):
        return len(self.experiences)

In [ ]:
class DQN_agent:


    def __init__(self, lr=0.0001 ,gamma=0.99, epsilon_params=(0.9,0.05,1000)):
        # Get cpu, gpu or mps device for training.
        self.device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        print(f"Using {self.device} device")
        self.pred_NN = CNN().to(self.device)
        self.target_NN = copy.deepcopy(self.pred_NN)
        self.target_NN.eval()
        self.gamma = gamma
        self.epsilon_start = epsilon_params[0]
        self.epsilon_end = epsilon_params[1]
        self.epsilon_decay = epsilon_params[2]
        self.optimizer = torch.optim.RMSprop(self.pred_NN.parameters(), lr=lr)
        self.steps_done = 0
        
    def predict(self, x):
        self.steps_done += 1
        return self.pred_NN.forward(x)
    
    def action(self, pred):
        eps = self.epsilon_end + (self.epsilon_start - self.epsilon_end) * math.exp(-1. * self.steps_done / self.epsilon_decay)
        return (
            random.randint(0, pred.size(dim=0) - 1)
            if random.random() < eps
            else torch.argmax(pred).item()
        )
    
    def train(self, experience_batch):
        loss_fn = nn.SmoothL1Loss()
        epoch_loss = 0
        states = torch.stack([experience_batch[i][0].squeeze(0) for i in range(len(experience_batch))]).to(self.device)
        actions = torch.tensor([experience_batch[i][1] for i in range(len(experience_batch))])
        rewards = torch.tensor([experience_batch[i][2] for i in range(len(experience_batch))]).to(self.device)#torch.tensor(experience_batch[:][2])
        next_states = torch.stack([experience_batch[i][3].squeeze(0)  for i in range(len(experience_batch))]).to(self.device)
        terminated = torch.tensor([not experience_batch[i][4] for i in range(len(experience_batch))]).to(self.device)
        y = self.estimated_value( rewards, next_states, terminated)
        self.optimizer.zero_grad()
        pred = self.pred_NN(states)
        indicies = torch.LongTensor(actions)
        indicies =indicies.unsqueeze(dim=0).T
        pred = pred.gather(1,indicies.to(self.device))
        loss = loss_fn(y, pred)
        loss.backward()
        self.optimizer.step()
        epoch_loss += loss.item()
        return epoch_loss
        
    def copy(self):
        self.target_NN.load_state_dict(self.pred_NN.state_dict())  
        
    def estimated_value(self, reward, next_state, done):
        with torch.no_grad():# vectorize it
                value_pred = self.target_NN.forward(next_state.to(self.device))
                action_pred =  self.pred_NN.forward(next_state.to(self.device))
                action_id = torch.argmax(action_pred,1).unsqueeze(1)
                value = value_pred.gather(1,action_id)
                done = done.unsqueeze(1)
                target = reward.unsqueeze(1) + self.gamma * torch.mul(value,done)
        return target
        

In [ ]:
def adjust_to_torch(t):
    t = t.unsqueeze(dim=0)
    return torch.movedim(t, 3, 1)

In [ ]:
#initialize environment
env = gym.make("ALE/Pacman-v5", render_mode="rgb_array")
env.seed(543)
env = gym.wrappers.AtariPreprocessing(env, screen_size=84, grayscale_obs=False, frame_skip=1, noop_max=40)
env = gym.wrappers.RecordVideo(env, './drive/MyDrive/videos_Pacman/videos', episode_trigger = lambda x: (x+1) % 100 == 0)# PATH
# env = NoopResetEnv(env, noop_max=30)
replay_buffer = Memory(10000)
torch.manual_seed(53407)
actions = range(env.action_space.n)
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
#hyperparams
max_steps = 9999
training_freq = 1
copying_freq = 20
batch_size = 64

#initialize agent
agent = DQN_agent(lr=2e-4,gamma=0.99)

training_session = 0
max_episode = 2000

#loops until max_time is reached
memory = Memory(10000)

total_steps = 0
last_rewards = []

episodes = []
losses = []
logger = {"episodes":[],"losses":[]}
for episode  in range(1,max_episode):
    #get first states
    state = env.reset()
    state = state[0]/255
    state = torch.Tensor(state)
    state = adjust_to_torch(state)
    #loops until experience_capacity is reached
    episode_reward = 0 
    episode_loss = 0
    steps = 0
    for i in range(1, max_steps):
        #predict q-values and choose action
        with torch.no_grad():
            pred = agent.predict(state.to(device))
        action = agent.action(pred)
        #get next states
        next_state, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        if i == max_steps - 1:
            print("Max steps reached.")
        next_state = adjust_to_torch(torch.tensor(next_state)/255)
        experience = [state, action, reward, next_state, terminated] 
        memory.add(experience)
        if terminated or truncated:
            break
        state = next_state      
        if len(memory) > 1000:
            experiences_train = memory.sample(batch_size)
            episode_loss += agent.train(experiences_train)
            training_session += 1
            steps += 1
        total_steps += 1
        steps = i
    #if episode % 100 == 0:
    if episode % copying_freq == 0:
        agent.copy()
    if episode % 2 == 0:
        logger["episodes"].append(episodes)
        logger["losses"].append(losses)
        episodes = []
        losses = []
    if episode % 100 == 0:
        with open('./drive/MyDrive/logs_Pacman/logger_ddqn.json', 'w') as fp:# PATH
            json.dump(logger, fp)
    if episode % 250 == 0:
      torch.save(agent.pred_NN.state_dict(), "./drive/MyDrive/logs_Pacman/pred.pt")
      torch.save(agent.target_NN.state_dict(), "./drive/MyDrive/logs_Pacman/target.pt")
    last_rewards.append(episode_reward)
    episodes.append(episode_reward)
    losses.append(episode_loss/steps)
    if steps > 0:
      print(f"Episode: {episode} Reward: {episode_reward} loss: {episode_loss/steps} last rewards: {sum(last_rewards)/len(last_rewards)}")
    


Using cuda device
Episode: 1 Reward: 1.0 loss: 0.0 last rewards: 1.0
Episode: 2 Reward: 0.0 loss: 0.0 last rewards: 0.5
Episode: 3 Reward: 4.0 loss: 0.0012774911345047328 last rewards: 1.6666666666666667
Episode: 4 Reward: 1.0 loss: 0.002371445245004053 last rewards: 1.5
Episode: 5 Reward: 2.0 loss: 0.0024679479681169805 last rewards: 1.6
Episode: 6 Reward: 1.0 loss: 0.0012800063932816705 last rewards: 1.5
Episode: 7 Reward: 1.0 loss: 0.0013041963436879803 last rewards: 1.4285714285714286
Episode: 8 Reward: 0.0 loss: 0.000923112272768729 last rewards: 1.25
Episode: 9 Reward: 8.0 loss: 0.0009164685336372097 last rewards: 2.0
Episode: 10 Reward: 15.0 loss: 0.002045460757747209 last rewards: 3.3
Episode: 11 Reward: 10.0 loss: 0.003176260713939571 last rewards: 3.909090909090909
Episode: 12 Reward: 24.0 loss: 0.004122880710293215 last rewards: 5.583333333333333
Episode: 13 Reward: 24.0 loss: 0.005273160559993137 last rewards: 7.0
Episode: 14 Reward: 16.0 loss: 0.005384668743970903 last rew

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode: 99 Reward: 20.0 loss: 0.007882533900412075 last rewards: 20.90909090909091
Moviepy - Building video /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-99.mp4.
Moviepy - Writing video /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-99.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-99.mp4
Episode: 100 Reward: 17.0 loss: 0.008281501198509747 last rewards: 20.87
Episode: 101 Reward: 26.0 loss: 0.015272476949361796 last rewards: 20.92079207920792
Episode: 102 Reward: 43.0 loss: 0.010550674766992368 last rewards: 21.137254901960784
Episode: 103 Reward: 24.0 loss: 0.009345255571653147 last rewards: 21.16504854368932
Episode: 104 Reward: 30.0 loss: 0.009019196991409574 last rewards: 21.25
Episode: 105 Reward: 26.0 loss: 0.008128472383079283 last rewards: 21.295238095238094
Episode: 106 Reward: 22.0 loss: 0.00882281840215398 last rewards: 21.30188679245283
Episode: 107 Reward: 19.0 loss: 0.008047997289694438 last rewards: 21.2803738317757
Episode: 108 Reward: 40.0 loss: 0.008150697856491654 last rewards: 21.453703703703702
Episode: 109 Reward: 20.0 loss: 0.008251138857432052 last rewards: 21.440366972477065
Episode: 110 Reward: 34.0 loss: 0.00877513286686016 last rewards:

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-199.mp4
Episode: 200 Reward: 41.0 loss: 0.007748076521961947 last rewards: 27.175
Episode: 201 Reward: 61.0 loss: 0.018962914066005178 last rewards: 27.34328358208955
Episode: 202 Reward: 48.0 loss: 0.010430110198149722 last rewards: 27.445544554455445
Episode: 203 Reward: 38.0 loss: 0.0113112394367962 last rewards: 27.497536945812808
Episode: 204 Reward: 43.0 loss: 0.011645529839266656 last rewards: 27.573529411764707
Episode: 205 Reward: 43.0 loss: 0.009951001791223588 last rewards: 27.648780487804878
Episode: 206 Reward: 46.0 loss: 0.010263410920024287 last rewards: 27.737864077669904
Episode: 207 Reward: 32.0 loss: 0.008916766595848847 last rewards: 27.758454106280194
Episode: 208 Reward: 24.0 loss: 0.007890139282731527 last rewards: 27.740384615384617
Episode: 209 Reward: 24.0 loss: 0.008011110088913344 last rewards: 27.72248803827751
Episode: 210 Reward: 30.0 loss: 0.00896055839557

...

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-1499.mp4
Episode: 1501 Reward: 35.0 loss: 0.08385932042285385 last rewards: 47.753497668221186
Episode: 1502 Reward: 27.0 loss: 0.06249807090467532 last rewards: 47.73968042609854
Episode: 1503 Reward: 103.0 loss: 0.06229414846074516 last rewards: 47.77644710578842
Episode: 1504 Reward: 96.0 loss: 0.05890859895617467 last rewards: 47.808510638297875
Episode: 1505 Reward: 45.0 loss: 0.05505264721340635 last rewards: 47.80664451827243
Episode: 1506 Reward: 48.0 loss: 0.03888388671950288 last rewards: 47.80677290836653
Episode: 1507 Reward: 58.0 loss: 0.038399910011018314 last rewards: 47.81353682813537
Episode: 1508 Reward: 22.0 loss: 0.04027032651276585 last rewards: 47.79641909814324
Episode: 1509 Reward: 68.0 loss: 0.03841305179389026 last rewards: 47.80980781974818
Episode: 1510 Reward: 29.0 loss: 0.03918729491086434 last rewards: 47.797350993377485
Episode: 1511 Reward: 30.0 loss: 0.0

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-1599.mp4
Episode: 1601 Reward: 20.0 loss: 0.16737076483134714 last rewards: 48.27482823235478
Episode: 1602 Reward: 61.0 loss: 0.11912432534533902 last rewards: 48.28277153558052
Episode: 1603 Reward: 19.0 loss: 0.10307580057112642 last rewards: 48.26450405489707
Episode: 1604 Reward: 32.0 loss: 0.07885629728250206 last rewards: 48.254364089775564
Episode: 1605 Reward: 17.0 loss: 0.07863100706508347 last rewards: 48.234890965732085
Episode: 1606 Reward: 24.0 loss: 0.06889836447255637 last rewards: 48.21980074719801
Episode: 1607 Reward: 63.0 loss: 0.06862090709633292 last rewards: 48.228998133167394
Episode: 1608 Reward: 102.0 loss: 0.07200127921650093 last rewards: 48.26243781094527
Episode: 1609 Reward: 20.0 loss: 0.05770007108630751 last rewards: 48.24487259167184
Episode: 1610 Reward: 94.0 loss: 0.07420754430696685 last rewards: 48.27329192546584
Episode: 1611 Reward: 23.0 loss: 0.09

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-1699.mp4
Episode: 1701 Reward: 55.0 loss: 0.18484919041477363 last rewards: 48.30099941211052
Episode: 1702 Reward: 98.0 loss: 0.1269491918372734 last rewards: 48.330199764982375
Episode: 1703 Reward: 41.0 loss: 0.12404380634409161 last rewards: 48.32589547856723
Episode: 1704 Reward: 23.0 loss: 0.11636965657959802 last rewards: 48.31103286384977
Episode: 1705 Reward: 101.0 loss: 0.1357108698950104 last rewards: 48.34193548387097
Episode: 1706 Reward: 18.0 loss: 0.12496002693487235 last rewards: 48.324150058616645
Episode: 1707 Reward: 173.0 loss: 0.12746331427738933 last rewards: 48.39718804920914
Episode: 1708 Reward: 39.0 loss: 0.11745938369754177 last rewards: 48.39168618266979
Episode: 1709 Reward: 26.0 loss: 0.10538364364265948 last rewards: 48.3785839672323
Episode: 1710 Reward: 32.0 loss: 0.11012843263297308 last rewards: 48.36900584795322
Episode: 1711 Reward: 30.0 loss: 0.10254

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-1799.mp4
Episode: 1801 Reward: 94.0 loss: 0.12249277633396585 last rewards: 48.915602443087174
Episode: 1802 Reward: 64.0 loss: 0.10072672094248784 last rewards: 48.92397336293008
Episode: 1803 Reward: 30.0 loss: 0.09295442766150493 last rewards: 48.9134775374376
Episode: 1804 Reward: 102.0 loss: 0.08270096345051846 last rewards: 48.94290465631929
Episode: 1805 Reward: 22.0 loss: 0.08698775582855806 last rewards: 48.92797783933518
Episode: 1806 Reward: 182.0 loss: 0.0934809861841061 last rewards: 49.001661129568106
Episode: 1807 Reward: 19.0 loss: 0.07836780712954428 last rewards: 48.98505810736027
Episode: 1808 Reward: 333.0 loss: 0.09577099531506428 last rewards: 49.14214601769911
Episode: 1809 Reward: 33.0 loss: 0.12009171366494578 last rewards: 49.13322277501382
Episode: 1810 Reward: 34.0 loss: 0.10874309604422626 last rewards: 49.12486187845304
Episode: 1811 Reward: 30.0 loss: 0.138

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/videos_Pacman/videos/rl-video-episode-1899.mp4
Episode: 1901 Reward: 22.0 loss: 0.20580927492282192 last rewards: 50.66070489216202
Episode: 1902 Reward: 177.0 loss: 0.15849724193021916 last rewards: 50.72712933753943
Episode: 1903 Reward: 94.0 loss: 0.1561177159945657 last rewards: 50.749868628481345
Episode: 1904 Reward: 39.0 loss: 0.11972872896831048 last rewards: 50.7436974789916
Episode: 1905 Reward: 47.0 loss: 0.10620847971301221 last rewards: 50.741732283464565
Episode: 1906 Reward: 24.0 loss: 0.12293951433772841 last rewards: 50.727701993704095
Episode: 1907 Reward: 181.0 loss: 0.09783435661682555 last rewards: 50.796014682747774
Episode: 1908 Reward: 93.0 loss: 0.09761857795003885 last rewards: 50.81813417190776
Episode: 1909 Reward: 88.0 loss: 0.08445605371768276 last rewards: 50.837611314824514
Episode: 1910 Reward: 94.0 loss: 0.08189437361022013 last rewards: 50.86020942408377
Episode: 1911 Reward: 32.0 loss: 0.0